In [1]:
import os

os.environ["SLACK_USER_TOKEN"] = os.getenv("SLACK_USER_TOKEN")

In [8]:
# Environment setup
import os

os.environ["SLACK_USER_TOKEN"] = os.getenv("SLACK_USER_TOKEN")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Basic imports (detailed setup cells 3-4 mein hai)
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import SlackToolkit
from langchain.agents import create_agent

toolkit = SlackToolkit()
slack_tools = toolkit.get_tools()
model = ChatOpenAI(model="gpt-4o-mini")

print(f"✅ Setup complete!")
print(f"📦 {len(slack_tools)} Slack tools loaded:")
for tool in slack_tools:
    print(f"   - {tool.name}")


✅ Setup complete!
📦 4 Slack tools loaded:
   - get_channelid_name_dict
   - get_messages
   - schedule_message
   - send_message


### Additional Practice Tools

Yahan kuch aur tools add kar rahe hain jo learning aur practice ke liye helpful hain:

In [9]:
# Additional Custom Tools for Practice
from langchain.tools import tool
from datetime import datetime
import json

@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems or calculations.")
def calculator(expression: str) -> str:
    """Evaluate mathematical expressions safely."""
    try:
        # Simple security - sirf numbers aur basic operators allow
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic math operations allowed"
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool("get_current_time", description="Gets the current date and time. Useful for scheduling or time-related queries.")
def get_current_time(timezone: str = "UTC") -> str:
    """Get current date and time."""
    now = datetime.now()
    return f"Current time ({timezone}): {now.strftime('%Y-%m-%d %H:%M:%S')}"

@tool("word_counter", description="Counts words, characters, or sentences in a given text. Useful for message analysis.")
def word_counter(text: str, count_type: str = "words") -> str:
    """Count words, characters, or sentences in text."""
    if count_type == "words":
        count = len(text.split())
        return f"Word count: {count}"
    elif count_type == "characters":
        count = len(text)
        return f"Character count: {count}"
    elif count_type == "sentences":
        count = len([s for s in text.split('.') if s.strip()])
        return f"Sentence count: {count}"
    else:
        return "Invalid count_type. Use 'words', 'characters', or 'sentences'"

@tool("text_transformer", description="Transforms text - can convert to uppercase, lowercase, or title case.")
def text_transformer(text: str, transform_type: str = "uppercase") -> str:
    """Transform text to different cases."""
    if transform_type == "uppercase":
        return text.upper()
    elif transform_type == "lowercase":
        return text.lower()
    elif transform_type == "title":
        return text.title()
    else:
        return f"Invalid transform_type. Use 'uppercase', 'lowercase', or 'title'"

@tool("simple_dictionary", description="Simple word definitions and synonyms. Basic dictionary lookup.")
def simple_dictionary(word: str) -> str:
    """Simple dictionary lookup (dummy data for practice)."""
    definitions = {
        "hello": "A greeting used to attract attention or begin a conversation.",
        "agent": "A person or thing that acts or has the power to act.",
        "tool": "A device or implement used to carry out a particular function.",
        "slack": "A messaging platform for team communication.",
        "python": "A high-level programming language known for its simplicity.",
    }
    word_lower = word.lower()
    if word_lower in definitions:
        return f"Definition of '{word}': {definitions[word_lower]}"
    else:
        return f"Word '{word}' not found in dictionary. Available words: {', '.join(definitions.keys())}"

# Sabhi custom tools ka list
custom_tools = [calculator, get_current_time, word_counter, text_transformer, simple_dictionary]

print(f"✅ {len(custom_tools)} custom tools defined:")
for tool in custom_tools:
    print(f"  - {tool.name}")

✅ 5 custom tools defined:
  - calculator
  - get_current_time
  - word_counter
  - text_transformer
  - simple_dictionary


In [10]:
# Slack tools + Custom tools ko combine karna
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import SlackToolkit
from langchain.agents import create_agent

# Slack toolkit se tools
toolkit = SlackToolkit()
slack_tools = toolkit.get_tools()

# Sabhi tools ko combine karo
all_tools = slack_tools + custom_tools

# Model setup
model = ChatOpenAI(model="gpt-4o-mini")

# Agent with all tools
agent_with_all_tools = create_agent(
    model=model,
    tools=all_tools,
)

print(f"✅ Agent created with {len(all_tools)} tools:")
print(f"  - {len(slack_tools)} Slack tools")
print(f"  - {len(custom_tools)} Custom tools")

✅ Agent created with 9 tools:
  - 4 Slack tools
  - 5 Custom tools


### Practice Examples - Custom Tools

Ab custom tools ke saath practice karein:

In [11]:
# Example 1: Calculator tool test
print("="*60)
print("Example 1: Calculator Tool")
print("="*60)
user_input = "What is 25 multiplied by 37, then add 100?"

for step in agent_with_all_tools.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Example 1: Calculator Tool
================================ Human Message =================================

What is 25 multiplied by 37, then add 100?
================================== Ai Message ==================================
Tool Calls:
  calculator (call_0ufcwpqrgMcIwSTfUXR1wwa5)
 Call ID: call_0ufcwpqrgMcIwSTfUXR1wwa5
  Args:
    expression: 25 * 37 + 100
================================= Tool Message =================================
Name: calculator

Result: 1025
================================== Ai Message ==================================

25 multiplied by 37, then added to 100, equals 1025.


In [12]:
# Example 2: Time + Text transformation
print("="*60)
print("Example 2: Multiple Custom Tools")
print("="*60)
user_input = "What is the current time? Also convert 'Hello World from LangChain' to uppercase."

for step in agent_with_all_tools.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Example 2: Multiple Custom Tools
================================ Human Message =================================

What is the current time? Also convert 'Hello World from LangChain' to uppercase.
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_AkNzpW3H6tgeteRoeeXDJ0qT)
 Call ID: call_AkNzpW3H6tgeteRoeeXDJ0qT
  Args:
  text_transformer (call_21kR0PSfslKi5LYgfEOdtBxf)
 Call ID: call_21kR0PSfslKi5LYgfEOdtBxf
  Args:
    text: Hello World from LangChain
    transform_type: uppercase
================================= Tool Message =================================
Name: text_transformer

HELLO WORLD FROM LANGCHAIN
================================== Ai Message ==================================

The current time is **2026-01-13 04:23:25** (UTC). 

The phrase converted to uppercase is **HELLO WORLD FROM LANGCHAIN**.


In [13]:
# Example 3: Word counter + Dictionary
print("="*60)
print("Example 3: Text Analysis Tools")
print("="*60)
user_input = "Count the words in this message: 'LangChain is amazing for building AI agents.' Also tell me the definition of 'agent'."

for step in agent_with_all_tools.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Example 3: Text Analysis Tools
================================ Human Message =================================

Count the words in this message: 'LangChain is amazing for building AI agents.' Also tell me the definition of 'agent'.
================================== Ai Message ==================================
Tool Calls:
  word_counter (call_nkIuOmElqCCrKMgFwwpSAye0)
 Call ID: call_nkIuOmElqCCrKMgFwwpSAye0
  Args:
    text: LangChain is amazing for building AI agents.
  simple_dictionary (call_z4G51ifbZTh0yYvpdvYlh953)
 Call ID: call_z4G51ifbZTh0yYvpdvYlh953
  Args:
    word: agent
================================= Tool Message =================================
Name: simple_dictionary

Definition of 'agent': A person or thing that acts or has the power to act.
================================== Ai Message ==================================

The message "LangChain is amazing for building AI agents." contains 7 words.

The definition of "agent" is: A person or thing that acts or has t

In [14]:
# Example 4: Slack + Custom Tools Combined
print("="*60)
print("Example 4: Slack + Custom Tools")
print("="*60)
user_input = "Calculate 123 + 456. Then send the result to the ai-assistant channel in Slack."

for step in agent_with_all_tools.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Example 4: Slack + Custom Tools
================================ Human Message =================================

Calculate 123 + 456. Then send the result to the ai-assistant channel in Slack.
================================== Ai Message ==================================
Tool Calls:
  calculator (call_gZ7sfeW9sPnHr0Y27DPilVMf)
 Call ID: call_gZ7sfeW9sPnHr0Y27DPilVMf
  Args:
    expression: 123 + 456
================================= Tool Message =================================
Name: calculator

Result: 579
================================== Ai Message ==================================
Tool Calls:
  send_message (call_QkTSptwscS97xFT969Umy1f1)
 Call ID: call_QkTSptwscS97xFT969Umy1f1
  Args:
    message: The result of 123 + 456 is 579.
    channel: ai-assistant
================================= Tool Message =================================
Name: send_message

Message sent: {'ok': True, 'channel': 'C0A83TH9KSP', 'ts': '1768260251.920979', 'message': {'user': 'U0A8P3C1S3B', 'type': 

In [15]:
# Example 5: Complex multi-step task
print("="*60)
print("Example 5: Complex Multi-Step Task")
print("="*60)
user_input = "Get current time, then count words in 'Hello from AI agent', transform it to title case, and define the word 'python'."

for step in agent_with_all_tools.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Example 5: Complex Multi-Step Task
================================ Human Message =================================

Get current time, then count words in 'Hello from AI agent', transform it to title case, and define the word 'python'.
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_kyzv5TdyYwyGLuUxFkTZ80Xe)
 Call ID: call_kyzv5TdyYwyGLuUxFkTZ80Xe
  Args:
    timezone: UTC
  word_counter (call_TD8kML15cPcdBhzyjxiukSjK)
 Call ID: call_TD8kML15cPcdBhzyjxiukSjK
  Args:
    text: Hello from AI agent
  text_transformer (call_MiexhZD910J0jMNSLAKSGeBG)
 Call ID: call_MiexhZD910J0jMNSLAKSGeBG
  Args:
    text: Hello from AI agent
    transform_type: title case
  simple_dictionary (call_L6KWOQ0AH2tymTTLvfpk6ARl)
 Call ID: call_L6KWOQ0AH2tymTTLvfpk6ARl
  Args:
    word: python
================================= Tool Message =================================
Name: simple_dictionary

Definition of 'python': A high-level programmi

### Original Slack Agent (Sirf Slack Tools)

Ye original agent hai jo sirf Slack tools use karta hai:

In [ ]:
# Original agent (sirf Slack tools) - ab ye cell 2 se updated ho gaya hai
# Isko comment out kar diya hai kyunki agent_with_all_tools better hai practice ke liye

# agent = create_agent(
#     model=model,
#     tools=slack_tools,
# )

# Test original agent
# user_input = "When was the #ai-assistant channel created?"
# user_input = "Send a friendly greeting to channel C0A83TH9KSP"
# user_input = "send a email to schudule a meeting with me at 10:00 AM PST only 30 minutes message just a dummpy for testing"

# for step in agent.stream(
#     {"messages": user_input},
#     stream_mode="values",
# ):
#     step["messages"][-1].pretty_print()

### Tips for Practice

**Tools ki list:**
1. **calculator** - Math calculations ke liye
2. **get_current_time** - Current date/time ke liye  
3. **word_counter** - Text analysis (words, characters, sentences count)
4. **text_transformer** - Text case conversion (uppercase, lowercase, title)
5. **simple_dictionary** - Word definitions lookup

**Practice karne ke liye suggestions:**
- Calculator tool ko different math expressions ke saath test karein
- Multiple tools ko ek hi query mein combine karein
- Slack tools + Custom tools ko saath mein use karein
- Complex multi-step tasks create karein

**Examples chahiye?** Cells 6-10 mein ready-made examples hain jo aap run kar sakte hain!